In [1]:
baseline = [
    './log.20190801-154135/',
    './log.archive/log.20190727-191215/',
    './log.20190801-024113/'
]

In [2]:
import os
import pickle
from pprint import pprint


global tmp_emr
def show_setup(log_path):
    with open(os.path.join(log_path, 'config.pkl'), 'rb') as fin:
        config = pickle.load(fin)
    source = config.pop('Source')
    display(Markdown('## Setup'))
    pprint(config)
    #global tmp_emr
    #tmp_emr = source.emr
    #%pycat tmp_emr

In [3]:
import json
from glob import glob
import re
from collections import defaultdict
import pandas as pd
from IPython.display import display, HTML, Markdown


def show_result(log_path):
    display(Markdown('## Result'))

    metrics_path = sorted(glob(os.path.join(log_path, 'metrics_epoch_*.json')), key=lambda p: os.path.getmtime(p)).pop()
    with open(metrics_path, 'r') as fin:
        metrics = json.load(fin)
        #pprint(metrics)

    def pop_regex(container, regex):
        if not isinstance(regex, re.Pattern):
            regex = re.compile(regex, re.MULTILINE | re.DOTALL)
        for key in list(container):
            match = regex.fullmatch(key)
            if match:
                yield match, container.pop(key)

    preserve_string = {'live_in': 'live-in',
                       'located_in': 'located-in',
                       'orgbase_on': 'orgbase-on',
                       'work_for': 'work-for'}

    def preserve(string):
        for k, v in preserve_string.items():
            string = string.replace(k, v)
        return string

    metrics_suffix = [
        'F1',
        'P',
        'R',
        'Accuracy',
        'loss',
    ]

    global_names = [
        'training_cpu_memory_MB',
        'training_duration',
        'training_epochs',
        'training_gpu_0_memory_MB',
        'training_gpu_1_memory_MB',
        'training_start_epoch',
        'best_epoch',
        'epoch',
        'peak_cpu_memory_MB',
        'peak_gpu_0_memory_MB',
        'peak_gpu_1_memory_MB',
    ]

    fields = [
        ('best', {'best'}),
        ('train', {'training', 'validation'}),
        ('inference', {'i'}),
    ]

    entities = ['people', 'organization', 'location', 'other']
    relations = ['kill', 'live-in', 'work-for', 'orgbase-on', 'located-in']

    def parse_fields(string_list):
        field_list = ['',] * 4
        for i, (field, field_set) in enumerate(fields):
            if field:
                for j, string in enumerate(list(string_list)):
                    if string in field_set:
                        field_list[i] = string
                        string_list.pop(j)
        field_list[-1] = string_list.pop().strip()
        assert not string_list
        return field_list
    entity_table = pd.DataFrame()
    best_epoch = metrics['best_epoch']

    global_data = {}
    for name in global_names:
        for match, value in pop_regex(metrics, name):
            global_data[match.string] = value
    global_table = pd.DataFrame.from_dict(global_data, orient='index')
    display(Markdown('### Global information'))
    display(HTML(global_table.to_html(header=False,)))

    for suffix in metrics_suffix:
        display(Markdown('#### ' + suffix))
        entity_dict = defaultdict(list)
        relation_dict = defaultdict(list)
        other_dict = defaultdict(list)
        entity_index = []
        relation_index = []

        total_dict = {}
        for match, value in pop_regex(metrics, r'(.*)(?<!-)-?{}'.format(suffix)):
            string_list = preserve(str(match[1])).split('_')
            field_list = parse_fields(string_list)
            total_dict[tuple(field_list)] = value

        for field_tuple, value in sorted(total_dict.items(), key=lambda k: k[0][-1]):
            field_list = list(field_tuple)
            index = field_list.pop()
            if index in entities:
                if index not in entity_index:
                    entity_index.append(index)
                entity_dict[tuple(field_list)].append(value)
            elif index in relations:
                if index not in relation_index:
                    relation_index.append(index)
                relation_dict[tuple(field_list)].append(value)
            elif not index:
                other_dict[tuple(field_list)] = value
            else:
                assert False, 'Not here, sir!'

        entity_table = pd.DataFrame.from_dict(entity_dict)
        entity_table = entity_table.append(entity_table.mean(numeric_only=True), ignore_index=True)
        entity_table.index = entity_index + ['mean',]

        relation_table = pd.DataFrame.from_dict(relation_dict)
        relation_table = relation_table.append(relation_table.mean(numeric_only=True), ignore_index=True)
        relation_table.index = relation_index + ['mean',]

        other_tabel = pd.DataFrame.from_dict(other_dict, orient='index')
        if entity_dict: display(HTML(entity_table.to_html()))
        if relation_dict: display(HTML(relation_table.to_html()))
        if other_dict: display(HTML(other_tabel.to_html(header=False,)))

    display(Markdown('### What\'s left'))
    left_tabel = pd.DataFrame.from_dict(metrics, orient='index')
    display(HTML(left_tabel.to_html(header=False,)))


In [4]:
log_path = baseline[0]
show_setup(log_path)
show_result(log_path)

## Setup

{'Data': {'relative_path': 'data/EntityMentionRelation',
          'train_path': 'conll04.corp_1_train.corp',
          'valid_path': 'conll04.corp_1_test.corp'},
 'Model': {'activation': LeakyReLU(negative_slope=0.01),
           'compact': Namespace('layers':[48]),
           'distance_emb_size': 8,
           'dropout': 0.3,
           'embedding_dim': 8,
           'graph': {'balance_factor': 1.5,
                     'focal_gamma': 2,
                     'inference_interval': 100,
                     'inference_training_set': False,
                     'label_smoothing': 0.01},
           'max_distance': 64,
           'ngram': 5,
           'pretrained_dims': Namespace('word':50),
           'pretrained_files': Namespace('word':data/glove.6B/glove.6B.50d.txt),
           'relconv': Namespace('layers':[None, None, None],'kernel_size':7),
           'relemb': Namespace('emb_size':256),
           'rnn': Namespace('layers':2,'bidirectional':True)},
 'Train': {'iterator': Namespac

## Result

### Global information

training_cpu_memory_MB,3776.28
training_duration,4:23:22.944098
training_epochs,99
training_gpu_0_memory_MB,22872
training_gpu_1_memory_MB,22228
training_start_epoch,0
best_epoch,11
epoch,99
peak_cpu_memory_MB,3776.28
peak_gpu_0_memory_MB,22872
peak_gpu_1_memory_MB,22228


...

#### Accuracy

#### loss

"(, training, )",0.007011
"(, validation, )",0.097879
"(best, validation, )",0.061502


### What's left

training_\n[ epoch ],99
validation_\n[ epoch ],99
best_validation_\n[ epoch ],11


In [5]:
log_path = baseline[1]
show_setup(log_path)
show_result(log_path)

## Setup

{'Data': {'relative_path': 'data/EntityMentionRelation',
          'train_path': 'conll04.corp_1_train.corp',
          'valid_path': 'conll04.corp_1_test.corp'},
 'Model': {'bidirectional': True,
           'dropout': 0.35,
           'embedding_dim': 8,
           'graph': {'balance_factor': 1.5,
                     'focal_gamma': 2,
                     'inference_interval': 100,
                     'inference_training_set': False,
                     'label_smoothing': 0.01},
           'ngram': 5,
           'pretrained_dims': Namespace('word':50),
           'pretrained_files': {'word': 'data/glove.6B/glove.6B.50d.txt'}},
 'Train': {'pretrained_files': Namespace('word':data/glove.6B/glove.6B.50d.txt),
           'trainer': {'batch': 8,
                       'epoch': 100,
                       'lr': 0.0001,
                       'patience': None,
                       'serialization_dir': 'log.20190727-191215',
                       'wd': 1e-05}}}


## Result

### Global information

training_cpu_memory_MB,3776.38
training_duration,4:21:45.178615
training_epochs,99
training_gpu_0_memory_MB,22872
training_gpu_1_memory_MB,22989
training_start_epoch,0
best_epoch,11
epoch,99
peak_cpu_memory_MB,3776.38
peak_gpu_0_memory_MB,22872
peak_gpu_1_memory_MB,22990


...

#### Accuracy

#### loss

"(, training, )",0.007837
"(, validation, )",0.098772
"(best, validation, )",0.062366


### What's left

training_\n[ epoch ],99
validation_\n[ epoch ],99
best_validation_\n[ epoch ],11


In [6]:
log_path = baseline[2]
show_setup(log_path)
show_result(log_path)

## Setup

{'Data': {'relative_path': 'data/EntityMentionRelation',
          'train_path': 'conll04.corp_1_train.corp',
          'valid_path': 'conll04.corp_1_test.corp'},
 'Model': {'activation': LeakyReLU(negative_slope=0.01),
           'compact': Namespace('layers':[48]),
           'distance_emb_size': 8,
           'dropout': 0.35,
           'embedding_dim': 8,
           'graph': {'balance_factor': 1.5,
                     'focal_gamma': 2,
                     'inference_interval': 100,
                     'inference_training_set': False,
                     'label_smoothing': 0.01},
           'max_distance': 64,
           'ngram': 5,
           'pretrained_dims': Namespace('word':50),
           'pretrained_files': Namespace('word':data/glove.6B/glove.6B.50d.txt),
           'relconv': Namespace('layers':[None, None, None],'kernel_size':7),
           'relemb': Namespace('emb_size':256),
           'rnn': Namespace('layers':2,'bidirectional':True)},
 'Train': {'iterator': Namespa

## Result

### Global information

training_cpu_memory_MB,3777.75
training_duration,4:22:39.658955
training_epochs,99
training_gpu_0_memory_MB,0
training_gpu_1_memory_MB,22990
training_start_epoch,0
best_epoch,11
epoch,99
peak_cpu_memory_MB,3777.75
peak_gpu_0_memory_MB,17922
peak_gpu_1_memory_MB,22990


...

#### Accuracy

#### loss

"(, training, )",0.007919
"(, validation, )",0.098383
"(best, validation, )",0.062082


### What's left

training_\n[ epoch ],99
validation_\n[ epoch ],99
best_validation_\n[ epoch ],11


In [7]:
log_path = './log.20190802-030244/'
show_setup(log_path)
show_result(log_path)

## Setup

{'Data': {'relative_path': 'data/EntityMentionRelation',
          'train_path': 'conll04.corp_1_train.corp',
          'valid_path': 'conll04.corp_1_test.corp'},
 'Model': {'activation': LeakyReLU(negative_slope=0.01),
           'compact': Namespace('layers':[48]),
           'distance_emb_size': 8,
           'dropout': 0.25,
           'embedding_dim': 8,
           'graph': {'balance_factor': 1.5,
                     'focal_gamma': 2,
                     'inference_interval': 100,
                     'inference_training_set': False,
                     'label_smoothing': 0.01},
           'max_distance': 64,
           'ngram': 5,
           'pretrained_dims': Namespace('word':50),
           'pretrained_files': Namespace('word':data/glove.6B/glove.6B.50d.txt),
           'relconv': Namespace('layers':[None, None, None],'kernel_size':7),
           'relemb': Namespace('emb_size':256),
           'rnn': Namespace('layers':2,'bidirectional':True)},
 'Train': {'iterator': Namespa

## Result

### Global information

training_cpu_memory_MB,3780.25
training_duration,4:21:48.144804
training_epochs,99
training_gpu_0_memory_MB,22872
training_gpu_1_memory_MB,22990
training_start_epoch,0
best_epoch,11
epoch,99
peak_cpu_memory_MB,3780.25
peak_gpu_0_memory_MB,22872
peak_gpu_1_memory_MB,22990


...

#### Accuracy

#### loss

"(, training, )",0.010130
"(, validation, )",0.085830
"(best, validation, )",0.061839


### What's left

training_\n[ epoch ],99
validation_\n[ epoch ],99
best_validation_\n[ epoch ],11
